In [ ]:
!pip install transformers accelerate -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.5 MB/s eta 0:00:00


In [ ]:
# bibliothèques nécessaires à la manipulation de données, au prétraitement et à la modélisation
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import re


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

# Load model and tokenizer
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-msa-did-nadi"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

# Get label mapping from the model config
id2label = model.config.id2label

# Dialect prediction function
def predict_dialect(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=128,  # Set reasonable max_length
        padding=True
    )
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
    return id2label.get(prediction, "UNKNOWN")  # Safe fallback


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
dataset = pd.read_csv("/content/Egyptian-tweets.csv")

In [ ]:
dataset.head(5)

,review,label
0,اكبر خطا ترتكبه ان تعامل الناس باخلاقك انت مش ...,negative
1,دائما اكره اخر ليله في كل مكان .,negative
2,يارب اللى يسرق تويتاتى يدخل النار .,negative
3,الاسراف فى تناول القهوة يسبب الوفاه .,negative
4,انا اتبهدلت من التراب النهارده. حاجة تقرف .,positive


In [ ]:
# Predict dialect and filter only Egyptian
dataset['dialect'] = dataset['review'].apply(predict_dialect)
#egyptian_only = dataset[dataset['dialect'] == 'EGY']


In [ ]:
# dataset.to_csv("/content/DATASET DIALECT.csv", index=False)

In [ ]:
# dataset.head(5)

In [ ]:
dataset['dialect'].value_counts()

,count
dialect,
Egypt,26630
United_Arab_Emirates,7055
Saudi_Arabia,4013
Algeria,459
Oman,366
Morocco,316
Iraq,186
Sudan,177
Syria,170


In [ ]:
# Filter dataset where dialect is Egyptian
egyptian_data = dataset[dataset['dialect'] == 'Egypt']
egyptian_data
# Display the distribution of the label column
#egyptian_data['LABEL'].value_counts()



,review,label,dialect
0,اكبر خطا ترتكبه ان تعامل الناس باخلاقك انت مش ...,negative,Egypt
2,يارب اللى يسرق تويتاتى يدخل النار .,negative,Egypt
3,الاسراف فى تناول القهوة يسبب الوفاه .,negative,Egypt
4,انا اتبهدلت من التراب النهارده. حاجة تقرف .,positive,Egypt
5,فى بنات بتلبـس اكســسوارات لدرجــة انك تحــس ا...,negative,Egypt
...,...,...,...
39994,المفروض يلغوا الامتحانات علشان نحتفل بالسنة ال...,negative,Egypt
39996,مش هقولك غير ان نص الضحك اللي ضحكته فى حياتي ك...,positive,Egypt
39997,ربنا يوفقك ويسهلك وان شاء الله تعدي الفترة دي ...,positive,Egypt
39998,مبسوطة اوى عملت طريقة مكرونة جديدة و هى دلوقتى...,positive,Egypt


In [ ]:
# prompt: write a code which count how many sample in each class, for only Egypt dialect

# Display the distribution of the label column for Egyptian data
print(egyptian_data['label'].value_counts())

label
negative            14631
positive            11994
positive ‎              2
positive​               1
negative​‏​‏            1
negative​‏​‏​‏​         1
Name: count, dtype: int64


In [ ]:
# prompt: write a code where can download the dataset of those label tweet of Egypt

# Filter the dataset to keep only rows where the predicted dialect is 'Egypt'
egyptian_data = dataset[dataset['dialect'] == 'Egypt']

# Save the filtered dataset to a new CSV file in the /content directory
egyptian_data.to_csv("/content/Egyptian_labeled_tweets.csv", index=False)

print("Egyptian labeled tweets dataset saved to /content/Egyptian_labeled_tweets.csv")

Egyptian labeled tweets dataset saved to /content/Egyptian_labeled_tweets.csv


In [ ]:
# prompt: write the code that uplode dataset, read it and count each sample in each class

from google.colab import files
import pandas as pd

# Upload the dataset file
uploaded = files.upload()

# Assuming the file is named 'your_dataset.csv' - replace with the actual filename you uploaded
for filename in uploaded.keys():
  print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')
  # Read the uploaded file into a pandas DataFrame
  try:
    dataset = pd.read_csv(filename)
    print(f"\nSuccessfully read '{filename}'.")
  except Exception as e:
    print(f"Error reading the file: {e}")
    continue # Skip to the next file if reading fails

  # Assuming the label column is named 'label' - adjust if necessary
  if 'LABEL' in dataset.columns:
    print("\nCounting samples per class:")
    print(dataset['LABEL'].value_counts())
  else:
    print("\n'label' column not found in the dataset. Please check the column name.")

# The subsequent code from the original notebook can continue to use the 'dataset' DataFrame if needed.
# However, the task was specifically to upload, read, and count samples per class.
# The rest of the provided code related to dialect prediction is not directly needed for this specific task but is kept below for continuity if desired.

# Keep the rest of the original code if it's intended to be used after uploading and reading
# dataset.head(5)
# # Predict dialect and filter only Egyptian
# dataset['dialect'] = dataset['review'].apply(predict_dialect)
# #egyptian_only = dataset[dataset['dialect'] == 'EGY']

# # dataset.to_csv("/content/DATASET DIALECT.csv", index=False)
# # dataset.head(5)
# dataset['dialect'].value_counts()
# # Filter dataset where dialect is Egyptian
# egyptian_data = dataset[dataset['dialect'] == 'Egypt']
# egyptian_data
# # Display the distribution of the label column
# #egyptian_data['LABEL'].value_counts()

# # Display the distribution of the label column for Egyptian data
# print(egyptian_data['label'].value_counts())

# # Filter the dataset to keep only rows where the predicted dialect is 'Egypt'
# egyptian_data = dataset[dataset['dialect'] == 'Egypt']

# # Save the filtered dataset to a new CSV file in the /content directory
# egyptian_data.to_csv("/content/Egyptian_labeled_tweets.csv", index=False)

# print("Egyptian labeled tweets dataset saved to /content/Egyptian_labeled_tweets.csv")

Saving DATASET.csv to DATASET (3).csv
User uploaded file "DATASET (3).csv" with length 3967538 bytes

Successfully read 'DATASET (3).csv'.

Counting samples per class:
LABEL
1    9000
0    9000
2    8191
Name: count, dtype: int64


In [11]:
import pandas as pd
import os

# # Step 1: Show current working directory
# print("Current directory:", os.getcwd())
# print("Files in the directory:", os.listdir())

# Step 2: Read dataset (replace with your actual filename)
filename = '/content/AR_sarcasm_train.csv'  # change this to your file name
df = pd.read_csv(filename)

# Step 3: Display a few rows
print("Dataset preview:")
print(df.head())


Dataset preview:
                                               TWEET  LABEL
0  "د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...      2
1  "مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...      2
2  “الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...      1
3  "قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...      2
4  "طبيب #المقاصة: إصابة #أحمد_الشيخ بسيطة تحتاج ...      1


In [12]:
# Replace 'label' with the correct column name for classes
label_column = 'LABEL'

if label_column in df.columns:
    print("\nNumber of samples per class:")
    print(df[label_column].value_counts())
else:
    print(f"Label column '{label_column}' not found. Available columns: {df.columns.tolist()}")



Number of samples per class:
LABEL
2    5266
1    4025
0    1907
Name: count, dtype: int64
